In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

### Section 6.1 2 Networks 
This section we need to 
1. create a graph
2. print out the `number of nodes` and `number of edges`
3. check the edges in the graph
4. print out the corresponding adjacency matrix
5. print out the `A == A.T`. If `A == A.T`, it means that the matrix is symmetric.


In [2]:
# 1. create a graph
G=nx.Graph()
G.add_edge(1,2)
G.add_edge(2,3)

# 2. print out & 3. check the edges
print('#Node: %i, #Edge: %i' % (G.number_of_nodes(), G.number_of_edges()))
print('Has 3-4 edge: %s' % G.has_edge(3,4))

G.add_nodes_from(range(1,7))

# 4. print out the corresponding adjacency matrix
print(nx.to_numpy_array(G).T)

# 5. print out the A==A.T
A = nx.to_numpy_array(G).T
A==A.T


#Node: 3, #Edge: 2
Has 3-4 edge: False
[[0. 1. 0. 0. 0. 0.]
 [1. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


array([[ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True],
       [ True,  True,  True,  True,  True,  True]])

### Section 6.3 Weighted Networks
1. add weight to edges and print the adjacency matrix

In [3]:
# 1. add weights to edges
G2=nx.Graph()
G2.add_edge(1,2, weight=3)
G2.add_edge(2,3,weight=2)
print(G2[2])
print(nx.to_numpy_array(G2).T)



{1: {'weight': 3}, 3: {'weight': 2}}
[[0. 3. 0.]
 [3. 0. 2.]
 [0. 2. 0.]]


### Section 6.4 Directed networks
1. create directed graph

In [8]:
# 1. directed graph, compared it to the book graph 6.2 in chapter 6.4
Di = nx.DiGraph()
Di.add_edges_from([(3,1), (2,3), (6,2),(5,6),(4,6),(5,4),(1,4),(3,5)])
print(nx.to_numpy_array(Di).T)

[[0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 1.]
 [1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 1. 0.]]


### The equation of the cocitation network is

$$ C_{ij} = \sum^n_{k=1} A_{ik} A_{jk} $$

### Section 6.4.1 Cocitation network
In this case we need to write a function to generate the cocitation network.
To answer questions in lab1:
1. _the sum should be zero but not, what did we miss?_
    In the beginning, I failed to generate zeros but I realized that we need to consider the `weight` of each edges into our network.
2. _Meaning of the neighbors for `Lienar Combination`? Print both_
    

In [38]:
### 1
# Read the gml file
G = nx.read_gml("./proofwikidefs_la.gml", 'name')
# print(G.adj)
def cocitation(G):
    """ 
    The algorithm to compute the cocitation index based on graphical approach
    By graphical approach, which means that we need to use networkx package to compute the edges between each pair of nodes, to find out the similarity between certain pair of nodes.
    Input:
        graph
    Output:
        cocitation results
    """
    # G.predecessors will return the a directed edge from m to n
    # rational: we have a list of the m to n data, then we need to see if they have overlap.
    # Define CG
    CG = nx.Graph()
    for ind, i in enumerate(G.nodes):
        for j in list(G.nodes)[ind:]:
            common_edges = [ele for ele in list(G.predecessors(i)) if ele in list(G.predecessors(j)) and ele in list(G.predecessors(j))]
            iw = [G.get_edge_data(ele, i)["weight"] for ele in common_edges]
            jw = [G.get_edge_data(ele, j)["weight"] for ele in common_edges]
            co_weight = np.dot(iw, jw)
            CG.add_edge(i, j, weight= co_weight)
    return CG
cocit_net = cocitation(G)
# test
a = nx.to_numpy_array(G).T
C1 = np.dot(a, a.T)
C2 = nx.to_numpy_array(cocit_net).T
Cdiff = C1-C2
print(f"Diff between C1 and C2 is {Cdiff.sum().sum()}")

ValueError: operands could not be broadcast together with shapes (256,256) (212,212) 

In [36]:
### 2
all_neighbors = list(nx.all_neighbors(cocit_net, "Linear Combination"))
all_weights = [cocit_net.get_edge_data(ele, "Linear Combination", default=0)["weight"] for ele in all_neighbors] 
for ind, i in enumerate(all_neighbors):
    print(f"Neighbor {ind} is {all_neighbors[ind]} with weight {all_weights[ind]}")

Neighbor 0 is Vector Space Monomorphism with weight False
Neighbor 1 is Vector Space Isomorphism with weight False
Neighbor 2 is Vector Space Epimorphism with weight False
Neighbor 3 is Vector Space Automorphism with weight False
Neighbor 4 is Vector Notation with weight False
Neighbor 5 is Vector (Euclidean Space) with weight 10.0
Neighbor 6 is Trilinear Mapping with weight False
Neighbor 7 is Transpose of Linear Transformation with weight False
Neighbor 8 is Trace (Linear Algebra)/Linear Transformation with weight False
Neighbor 9 is Symmetric Mapping with weight False
Neighbor 10 is Standard Ordered Basis/Vector Space with weight False
Neighbor 11 is Standard Basis/Vector Space with weight False
Neighbor 12 is Spanning Set with weight False
Neighbor 13 is Simultaneous Equations/Solution Set with weight False
Neighbor 14 is Simultaneous Linear Equations/Matrix Representation with weight False
Neighbor 15 is Simultaneous Linear Equations with weight False
Neighbor 16 is Simultaneous E